In [1]:
#!import ../Settings/AppConfig.cs

AppConfig.aoaiSettings apiConfig = AppConfig.GetSecrets();

// Get AOAI values
string mydeploymentName = apiConfig.deployTurbo;
string myendpoint = apiConfig.endpoint;
string myAPIKey = apiConfig.APIKey;

//NOTE: If you run the below writeline command, be sure to clear the output before pushing to repo!!
//Console.WriteLine(mydeploymentName + " & " + myendpoint + " & " + myAPIKey);

In [2]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.0.1"

Installed Packages Microsoft.SemanticKernel, 1.0.1

In [3]:
using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

//Create Kernel builder
var builder = Kernel.CreateBuilder();

// Configure AI service credentials used by the kernel
builder.AddAzureOpenAIChatCompletion(mydeploymentName, myendpoint, myAPIKey);


var SK = builder.Build();

In [4]:
// Batman Chat Plugin directory path
var chatPluginDirectoryPath = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "plugins", "ChatPlugin");

// Create the Chat Function from the Plugins Directory
var batmanChatFunction = SK.CreatePluginFromPromptDirectory(chatPluginDirectoryPath,"BatmanChat");


In [ ]:
var history = "";
var arguments = new KernelArguments()
{
    ["history"] = history
};

var input = "Who is your favorite villain?";
arguments["input"] = input;

In [9]:
using Microsoft.DotNet.Interactive;
using Microsoft.SemanticKernel.ChatCompletion;

// Batman Chat Plugin directory path
var chatPluginDirectoryPath = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "plugins", "ChatPlugin");

// create Chat Function from plugin dir
var batmanChatFunction = SK.CreatePluginFromPromptDirectory(chatPluginDirectoryPath,"BatmanChat");

// Create chat history
ChatHistory history = [];
var interactiveinput = "";

// Start the chat loop
while (true)
{
    // Get user input
    interactiveinput = await GetInputAsync("Pick a number.");    

    // Get chat response
    var batman_answer = SK.InvokeStreamingAsync<StreamingChatMessageContent>(batmanChatFunction["BatmanChat"], 
                        new() { { "input", interactiveinput },
                                { "history", string.Join("\n", history.Select(x => x.Role + ": " + x.Content)) }});


    // Stream the response
    string message = "";
    Console.Write("Fan question: " + interactiveinput+". ");
    await foreach (var chunk in batman_answer)
    {
        if (chunk.Role.HasValue) Console.Write(chunk.Role + " > ");
        message += chunk;
        Console.Write(chunk);
    }
    Console.WriteLine();

    // Append to history    
    history.AddUserMessage(interactiveinput!);
    history.AddAssistantMessage(message);
}

Fan question: where do you work. assistant > I work in Gotham City, protecting its citizens from crime and injustice.
Fan question: do you work with the police. assistant > Yes, I work closely with Commissioner Gordon and the Gotham City Police Department to keep the city safe.
Fan question: who is your favorite villian. assistant > I do not have a favorite villain. All criminals must be brought to justice.
Fan question: do you love cat woman. assistant > I do not have romantic feelings towards any criminals, including Catwoman. My focus is on protecting Gotham City and upholding justice.


Error: Input request cancelled

Error: System.Exception: Input request cancelled
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, Boolean isPassword, String typeHint, String valueName) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 72
   at Microsoft.DotNet.Interactive.Kernel.GetInputAsync(String prompt, String typeHint, String valueName) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive\Kernel.Static.cs:line 46
   at Submission#11.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)